In [1]:
import torch
import torchvision
import numpy as np 
import random
import math
import matplotlib.pyplot as plt

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
file = open('../input/ibug-300w-large-face-landmark-dataset/ibug_300W_large_face_landmark_dataset/helen/testset/2986046144_1.pts')
points = file.readlines()[3:-1]

landmarks = []

for point in points:
    x, y = point.split(' ')
    landmarks.append([np.floor(float(x)), np.floor(float(y[:-1]))])

landmarks = np.array(landmarks)

plt.figure(figsize=(10, 10))
plt.imshow(Image.open('../input/ibug-300w-large-face-landmark-dataset/ibug_300W_large_face_landmark_dataset/helen/testset/2986046144_1.jpg'))
plt.scatter(landmarks[:, 0], landmarks[:, 1], s=40)
plt.show()

In [3]:
import cv2

In [7]:
from dataset import FaceLandmarksDataset
from transforms import Transforms
from model import ResNet50
from torchvision import models

In [5]:
FaceDataset = FaceLandmarksDataset(Transforms())

FileNotFoundError: [Errno 2] No such file or directory: '../input/ibug-300w-large-face-landmark-dataset/ibug_300W_large_face_landmark_dataset/labels_ibug_300W_train.xml'

In [8]:
weights = models.ResNet50_Weights.DEFAULT

In [23]:
model = models.resnet50(weights=weights)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [17]:
import torch.nn as nn

In [24]:
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.fc = nn.Linear(model.fc.in_features, 136)

In [25]:
model.to(device)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [10]:
len_test_set = int(0.2 * (len(FaceDataset)))
len_train_set = len(FaceDataset) - len_test_set

NameError: name 'FaceDataset' is not defined

In [ ]:
train_dataset, test_dataset = torch.utils.data.random_split(FaceDataset, [len_train_set, len_test_set])

In [13]:
from torch.utils.data import DataLoader
import os
import torch.nn as nn

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=os.cpu_count())
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=os.cpu_count())

In [14]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)

In [ ]:
def train_test(model, epochs, loss_fn, optimizer, train_loader, test_loader):
    torch.autograd.set_detect_anomaly(True)
    loss_min = np.inf

    train_loss = []
    test_loss = []

    for epoch in range(epochs):
        loss_train = 0
        loss_test = 0
        running_loss = 0

        model.train()
        for batch, (image, landmarks) in enumerate(train_loader):
            image.to(device)
            landmarks = landmarks.view(landmarks.shape[0], -1).to(device)

            predictions = model(image)
            loss_train_step = loss_fn(predictions, landmarks)

            optimizer.zero_grad()
            loss_train_step.backward()
            optimizer.step()

            loss_train += loss_train_step.item()
            running_loss = loss_train / batch

            print(f'{batch}, {len(train_loader)}, {running_loss}, train')

        model.eval()
        with torch.no_grad():
            for batch, (image, landmarks) in enumerate(test_loader):
                image.to(device)
                landmarks = landmarks.view(landmarks.shape[0], -1).to(device)
                
                predictions = model(image)

                loss_test_step = loss_fn(predictions, landmarks)
                loss_test += loss_test_step.item()
                running_loss = loss_test / batch

                print(f'{batch}, {len(test_loader)}, {running_loss}, test')

        loss_train /= len(train_loader)
        loss_test /= len(test_loader)

        print('\n--------------------------------------------------')
        print('Epoch: {}  Train Loss: {:.6f}  Test Loss: {:.6f}'.format(epoch, loss_train, loss_test))
        print('--------------------------------------------------')

        train_loss.append(loss_train)
        test_loss.append(loss_test)

        if loss_test < loss_min:
            loss_min = loss_test
            torch.save(model.state_dict(), '/progress.pth')
            print("\nMinimum Test Loss of {:.6f} at epoch {}/{}".format(loss_min, epoch, epochs))
            print('Model Saved\n')

In [ ]:
def show_res(model, test_loader):
    with torch.no_grad():
        network = model
        network.to(device)
        network.load_state_dict(torch.load('/progress.pth'))
        network.eval()

        image, landmarks = next(iter(test_loader))

        image.to(device)
        landmarks = (landmarks + 0.5) * 224

        predictions = (network(image).cpu() + 0.5) * 224
        predictions = predictions.view(-1.68, 2)

        plt.figure(figsize=(10, 40))

        for img_num in range(4):
            plt.subplot(1,4,img_num+1)
            plt.imshow(image[img_num].cpu().numpy().transpose(1,2,0).squeeze(), cmap='gray')
            plt.scatter(predictions[img_num,:,0], predictions[img_num,:,1], c = 'r', s = 5)
            plt.scatter(landmarks[img_num,:,0], landmarks[img_num,:,1], c = 'g', s = 5)